In [1]:
import requests
from datetime import datetime, timedelta
from pprint import pprint

In [2]:
# 측정소별 실시간 측정정보 조회 (매시 15분 내외 업데이트)
base_url = "http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty"
with open('keys/에어코리아api3.txt') as file:
    service_key = file.read()       # 발급받은 에어코리아 API 키 입력

In [4]:
# 웹 요청시 같이 전달될 데이터 = 요청 메시지
params = {
        'serviceKey' : service_key,
        'returnType' : 'JSON',
        'numOfRows' : 30,
        'pageNo' : 1,
        'stationName' : "과천동", # 시도 이름
        'dataTerm' : 'DAILY',
        'ver' : "1.4" # 오퍼레이션 버전
    }

res = requests.get(url=base_url , params=params)
# print(res.status_code, type(res.text), res.url)
# print()
# print(res.text)

# 응답 데이터 정리
from pprint import pprint  # 구조있는 데이터를 더 편하게 보여줌
data = res.json() # json.loads(res.text)와 같은 기능
data = data['response']['body']['items']
pprint(data)

[{'coFlag': '통신장애',
  'coGrade': None,
  'coValue': '-',
  'dataTime': '2023-11-28 11:00',
  'khaiGrade': None,
  'khaiValue': '-',
  'mangName': '도시대기',
  'no2Flag': '통신장애',
  'no2Grade': None,
  'no2Value': '-',
  'o3Flag': '통신장애',
  'o3Grade': None,
  'o3Value': '-',
  'pm10Flag': '통신장애',
  'pm10Grade': '1',
  'pm10Grade1h': None,
  'pm10Value': '-',
  'pm10Value24': '27',
  'pm25Flag': '통신장애',
  'pm25Grade': None,
  'pm25Grade1h': None,
  'pm25Value': '-',
  'pm25Value24': '-',
  'so2Flag': '통신장애',
  'so2Grade': None,
  'so2Value': '-',
  'stationCode': '131202',
  'stationName': '과천동'},
 {'coFlag': None,
  'coGrade': '1',
  'coValue': '0.4',
  'dataTime': '2023-11-28 10:00',
  'khaiGrade': None,
  'khaiValue': '-',
  'mangName': '도시대기',
  'no2Flag': None,
  'no2Grade': '1',
  'no2Value': '0.013',
  'o3Flag': None,
  'o3Grade': '1',
  'o3Value': '0.027',
  'pm10Flag': None,
  'pm10Grade': '1',
  'pm10Grade1h': '1',
  'pm10Value': '16',
  'pm10Value24': '27',
  'pm25Flag': '통신장애',
 

In [71]:
# 완성코드

def get_air_quality(station_Name):
    
    # 웹 요청시 같이 전달될 데이터 = 요청 메시지
    params = {
        'serviceKey' : service_key,
        'returnType' : 'JSON',
        'numOfRows' : 30,
        'pageNo' : 1,
        'stationName' : station_Name, # 시도 이름
        'dataTerm' : 'DAILY',
        'ver' : "1.4" # 오퍼레이션 버전
    }

    res = requests.get(url=base_url , params=params)

    # 응답 데이터 정리
    from pprint import pprint  # 구조있는 데이터를 더 편하게 보여줌
    data = res.json() # json.loads(res.text)와 같은 기능
    data = data['response']['body']['items']
    
    # 데이터를 날짜와 시간에 대한 기준으로 정렬
    sorted_data = sorted(data, key=lambda x: x['dataTime'], reverse=True)

    # 최신 데이터 선택
    latest_data = sorted_data[0]
    
    # 최종 데이터 생성
    results = {
        '날짜': latest_data['dataTime'],
        '이름': latest_data['stationName'],
        '측정망 정보': latest_data['mangName'],
        '아황산가스 농도': latest_data['so2Value'] + "ppm",
        '일산화탄소 농도': latest_data['coValue'] + "ppm",
        '오존 농도': latest_data['o3Value'] + "ppm",
        '이산화질소 농도': latest_data['no2Value'] + "ppm",
        '미세먼지(PM10) 농도': latest_data['pm10Value'] + "ug/m³",
        '초미세먼지(PM2.5) 농도': latest_data['pm25Value'] + "ug/m³"
    }

    # pprint(results)
    return results
    
# 테스트 코드
if __name__ == "__main__":
    print(get_air_quality('대신동'))

{'날짜': '2023-11-25 13:00', '이름': '대신동', '측정망 정보': '도시대기', '아황산가스 농도': '0.002ppm', '일산화탄소 농도': '0.3ppm', '오존 농도': '0.030ppm', '이산화질소 농도': '0.013ppm', '미세먼지(PM10) 농도': '15ug/m³', '초미세먼지(PM2.5) 농도': '7ug/m³'}
